# MVD 11. cvičení


## 1. část - Collaborative Filtering (item-item)

Vytvořte si matici podle příkladu v přednášce na item-item přístup na snímku 24. Využijte centered cosine similarity pro výpočet podobností stejně jako v přednášce a vyberte dva nejbližší sousedy. Ověřte, že je výsledek v přednášce správný a implementujte funkci tak, aby bylo možné jednoduše spočítat i libovolné další vyhodnocení.

In [1]:
import torch
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-5)

In [2]:
def item_item(matrix, k, product):
    mean_r = matrix.sum(1)/(matrix != 0).sum(1)
    norm = matrix - mean_r[:,None]
    norm_matrix = torch.where(matrix != 0, norm, 0)
    cos_sim = cos(norm_matrix[product[0],:], norm_matrix).squeeze()
    _,idx = torch.topk(cos_sim, k+1)
    idx = idx[1:]
    return (cos_sim[idx] * matrix[idx, product[1]]).sum() / cos_sim[idx].sum() + 1e-6


In [3]:
matrix = torch.tensor([[1,0,3,0,0,5,0,0,5,0,4,0],
                       [0,0,5,4,0,0,4,0,0,2,1,3],
                       [2,4,0,1,2,0,3,0,4,3,5,0],
                       [0,2,4,0,5,0,0,4,0,0,2,0],
                       [0,0,4,3,4,2,0,0,0,0,2,5],
                       [1,0,3,0,3,0,0,2,0,0,4,0]])
print(item_item(matrix, 2, [0,4]))

tensor(2.5864)


## Bonus - Content-based Filtering

Stáhněte si Kaggle dataset [Spotify Recommendation system](https://www.kaggle.com/bricevergnou/spotify-recommendation). Z datasetu budete potřebovat:

- data.csv = příznaky k jednotlivým skladbám + příznak liked pro klasifikaci

Úkolem je:

1. Načíst data z csv.
2. Vytvořit train (90 %) a test (10 %) split pomocí knihovny [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).
3. Vytvořit model logistické regrese (vlastní implementace / využití knihovny sklearn nebo jiné).
4. Vyhodnotit data na testovací sadě (např. metoda score u LogisticRegression).

**Skóre pro uznání úlohy musí být vyšší než 89 %.**

Dobrovolné: vytvořit graf predikovaných vs aktuálních hodnot

Získání bodu navíc: Využijte i další algoritmy pro binární klasifikaci.

In [142]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [162]:
# 1. Načíst data z csv
df = pd.read_csv("data/data.csv", header = 0)
print(len(df))
y = df['liked']
x = df.loc[:, df.columns != 'liked']

195


In [165]:
# 2. Vytvořit train (90 %) a test (10 %) split pomocí knihovny sklearn.
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.9)

In [158]:
# 3. Vytvořit model logistické regrese (vlastní implementace / využití knihovny sklearn nebo jiné).
logReg = LogisticRegression(max_iter = 200).fit(x_train, y_train)
score = logReg.score(x_test, y_test)
print(score)

0.95


In [179]:
result = y_test.to_frame()
result = result.rename(columns={result.columns[0]: "true_label"})
result['predicted_label'] = logReg.predict(x_test)
result['correct'] = (result['true_label']==result['predicted_label']).astype(int)
print(result)


     true_label  predicted_label  correct
11            1                1        1
150           1                1        1
102           1                1        1
149           1                1        1
152           1                1        1
128           0                0        1
122           1                1        1
134           1                1        1
91            0                0        1
125           1                1        1
87            0                0        1
40            0                0        1
182           1                1        1
140           1                1        1
132           0                1        0
42            1                1        1
189           0                0        1
181           1                1        1
161           1                1        1
71            1                1        1


In [150]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
score = knn.score(x_test, y_test)
print(score)

0.8


In [151]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

sgd = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3))
sgd.fit(x_train, y_train)
score =sgd.score(x_test, y_test)
print(score)

0.9


In [152]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)
score = clf.score(x_test, y_test)
print(score)

0.85


In [157]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=20, max_depth=9)
rf.fit(x_train, y_train)
score = rf.score(x_test, y_test)
print(score)

0.9
